In [2]:
:dep ndarray = "0.15.0"
:dep rand = "0.6.5"
:dep plotly = "0.6.0"
:dep itertools-num
:dep nalgebra
:dep num-traits

In [3]:
extern crate rand;
use rand::distributions::{Normal, Distribution};

In [4]:
extern crate plotly;
use plotly::{Plot, Layout, Scatter};

# Generate data

In [5]:
struct RandomMotionParameters {
    initial_state : ndarray::Array1<f64>,
    motion_noise : f64,
    measurement_noise : f64
}

fn generate_data(point_count: i64, parameters: &RandomMotionParameters, time_delta: f64)
    -> (Vec<ndarray::Array1<f64>>, Vec<ndarray::Array1<f64>>) {
    let mut state: ndarray::Array1<f64> = parameters.initial_state.clone();
    let motion_distribution = Normal::new(0., parameters.motion_noise);
    let measurement_distribution = Normal::new(0., parameters.motion_noise);
    let mut measurements: Vec<ndarray::Array1<f64>> = Vec::new();
    let mut states: Vec<ndarray::Array1<f64>> = Vec::new();
    for _ in 0..point_count {
        let acceleration = motion_distribution.sample(&mut rand::thread_rng());
        state = ndarray::arr1(&[
            state[0] + state[1] * time_delta + acceleration * time_delta.powi(2) / 2.,
            state[1] + acceleration * time_delta
        ]);
        let measurement:ndarray::Array1<f64> = ndarray::arr1(&[
            state[0] + measurement_distribution.sample(&mut rand::thread_rng())
        ]);
        measurements.push(measurement.clone());
        states.push(state.clone());
    }
    (measurements, states)
}


In [6]:
let time_delta = 0.1;
let points_count:i64 = 500;
let times: Vec<f64> =  (0..points_count).map(|x: i64| -> f64 {x as f64 * time_delta}).collect();
let initial_state : ndarray::Array1<f64> = ndarray::arr1(&[0., 0.]);
let parameters = RandomMotionParameters{
    initial_state: initial_state,
    measurement_noise: 1.,
    motion_noise: 1.
};
let result:(Vec<ndarray::Array1<f64>>, Vec<ndarray::Array1<f64>>) = generate_data(points_count, &parameters, time_delta);
let measurements: Vec<ndarray::Array1<f64>> = result.0;
let states: Vec<ndarray::Array1<f64>> = result.1;
let measurement_data: Vec<f64> = measurements.iter().map(|x| -> f64 {x[0]}).collect();
let ground_truth_data: Vec<f64> = states.iter().map(|x| -> f64 {x[0]}).collect();

## Plot

In [7]:
let trace1 = Scatter::new(times.clone(), ground_truth_data.clone());
let trace2 = Scatter::new(times.clone(), measurement_data.clone());
let mut plot = Plot::new();
plot.add_trace(trace1);
plot.add_trace(trace2);
plot.notebook_display();

## Filter state

In [8]:
#[path="C:\\Users\\Mikhail\\Documents\\kalman-rust\\src\\kalman.rs"]
mod kalman;

In [9]:
extern crate nalgebra as na;

In [10]:
use na::{Vector1, Vector2, Matrix2};

In [11]:
let measurement_vectors: Vec<Vector1<f64>> = measurement_data
    .iter()
    .map(|x| -> Vector1<f64>{Vector1::from_vec(vec![*x])})
    .collect();

In [16]:
let kalman_filter: kalman::KalmanFilter<f64, na::U2, na::U1> = kalman::make_simple_kalman_filter(0.1, 1.0, 1.0);
let inital_state: kalman::KalmanState<f64, na::U2> = kalman::KalmanState{
    state: Vector2::from_vec(vec![0., 0.]),
    covariance: Matrix2::from_vec(vec![1., 0., 0., 1.])
};
let filtred_states: Vec<kalman::KalmanState<f64, na::U2>> = kalman_filter.filter_states(&measurement_vectors, 
    inital_state.clone(), 1e-8);
let filtred_data: Vec<f64> = filtred_states
    .iter()
    .map(|x| -> f64{x.state[0]})
    .collect();

In [17]:
let trace1 = Scatter::new(times.clone(), ground_truth_data.clone());
let trace2 = Scatter::new(times.clone(), measurement_data.clone());
let trace3 = Scatter::new(times.clone(), filtred_data.clone());
let mut plot = Plot::new();
plot.add_trace(trace1);
plot.add_trace(trace2);
plot.add_trace(trace3);
plot.notebook_display();

In [14]:
use std::time::{Duration, Instant};

In [22]:
let mut filtred_states1: Vec<kalman::KalmanState<f64, na::U2>> = Vec::new();
let start = Instant::now();
for i in 1..100000{
    filtred_states1 = kalman_filter.filter_states(&measurement_vectors, 
    inital_state.clone(), 1e-8);
}
let duration = start.elapsed();
duration

5.1496663s